In [60]:
import pandas as pd
import random

In [31]:
df = pd.read_csv("BankWages.csv", index_col=0)
df.head()

,job,education,gender,minority
1,manage,15,male,no
2,admin,16,male,no
3,admin,12,female,no
4,admin,8,female,no
5,admin,15,male,no


In [32]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

categorized_labels = ["job", "gender", "minority"]
for cl in categorized_labels:
    encoded_label = le.fit_transform(df[cl])
    df.drop(cl, axis=1, inplace=True)
    df[cl] = encoded_label
    
df.head()

,education,job,gender,minority
1,15,2,1,0
2,16,0,1,0
3,12,0,0,0
4,8,0,0,0
5,15,0,1,0


In [34]:
df.groupby("minority").describe()

education                                                      job  \
             count       mean       std  min   25%   50%   75%   max  count   
minority                                                                      
0            370.0  13.694595  2.942304  8.0  12.0  15.0  16.0  21.0  370.0   
1            104.0  12.769231  2.555142  8.0  12.0  12.0  15.0  19.0  104.0   

                    ...           gender                                     \
              mean  ...  75%  max  count      mean       std  min  25%  50%   
minority            ...                                                       
0         0.470270  ...  1.0  2.0  370.0  0.524324  0.500084  0.0  0.0  1.0   
1         0.201923  ...  0.0  2.0  104.0  0.615385  0.488860  0.0  0.0  1.0   

                    
          75%  max  
minority            
0         1.0  1.0  
1         1.0  1.0  

[2 rows x 24 columns]

In [50]:
from sklearn.model_selection import train_test_split

X = df.drop("minority", axis=1).values
y = df["minority"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [6]:
def identity(x):
    return x

In [84]:
class Neuron():
    def __init__(self, activation_function):
        self.w = random.random()
        self.b = random.random()
        self.a_f = activation_function
    
    def call(self, *inputs):
        induced_local_field = sum([i * self.w + self.b for i in inputs])
        output = self.a_f(induced_local_field)
        return output
    
    def update(self, delta_w, delta_b):
        self.w += delta_w
        self.b += delta_b
        
    @property
    def weights(self):
        return self.w, self.b

In [127]:
class NeuralLayer():
    def __init__(self, neuron_count, activation_function):
        self.neurons = [Neuron(activation_function) for _ in range(neuron_count)]
        
    def calculate_outputs(self, x_input):
        return [neuron.call(x_input) for neuron in self.neurons]
    
    @property
    def weights(self):
        return [neuron.weights for neuron in self.neurons]
        

In [128]:
class NNModel():
    def __init__(self, *layers):
        self.layers = layers
        
        
    def fit(self, x_input, y_output):
        layer_output = self.layers[0].calculate_outputs(x_input)
        return layer_output
        
    @property
    def weights(self):
        return [neural_layer.weights for neural_layer in self.layers]

In [136]:
n = NNModel(
    NeuralLayer(4,identity),
    NeuralLayer(4,identity),
)



[array([4.6878209 , 0.56245158, 0.56245158]),
 array([10.91393249,  0.34973919,  0.34973919]),
 array([10.25012407,  0.93124864,  0.93124864]),
 array([3.63896966, 0.29026332, 0.29026332])]

In [140]:
n.fit(X_train[0], y_train[0])

[array([4.6878209 , 0.56245158, 0.56245158]),
 array([10.91393249,  0.34973919,  0.34973919]),
 array([10.25012407,  0.93124864,  0.93124864]),
 array([3.63896966, 0.29026332, 0.29026332])]